## AI Game playing project

In [8]:
# prompt: generate chess game

import random

class ChessGame:
    def __init__(self, black_king_pos):
        self.board = [['.' for _ in range(8)] for _ in range(8)]
        self.board_black = [['.' for _ in range(8)] for _ in range(8)]
        self.current_player = 'white'
        self.black_king = self.position_to_index(black_king_pos)
        self.pieces_black = []
        self.pieces_white = []
        self.possible_moves_black = []
        self.possible_moves_white = []
        self.checked = False
        self.stalemated = False
        self.checkmated = False
        self.score = 0
        self.add_piece('k'+black_king_pos)

    def print_board(self):
        print('  +---+---+---+---+---+---+---+---+')
        for row in range(8):
            print(8-row, end=' | ')
            for col in range(8):
                print(self.board[row][col], end=' | ')
            print('\n  +---+---+---+---+---+---+---+---+')
        print('    a   b   c   d   e   f   g   h')

    def print_black_board(self):
        print('  +---+---+---+---+---+---+---+---+')
        for row in range(8):
            print(8-row, end=' | ')
            for col in range(8):
                print(self.board_black[row][col], end=' | ')
            print('\n  +---+---+---+---+---+---+---+---+')
        print('    a   b   c   d   e   f   g   h')
    
    def position_to_index(self, inp):
        if len(inp) == 2:
            col = ord(inp[0].lower())-97
            row = 8-int(inp[1])
            return tuple([row,col])
        elif len(inp) == 3:
            piece = inp[0]
            col = ord(inp[1].lower())-97
            row = 8-int(inp[2])
            return tuple([piece,row,col])
        else:
            piece = inp[0]
            col1 = ord(inp[1].lower())-97
            row1 = 8-int(inp[2])
            col2 = ord(inp[3].lower())-97
            row2 = 8-int(inp[4])
            piece2 = piece
            if len(inp) == 6:
                piece2 = inp[5]
        if self.current_player == 'white':
            piece = piece.upper()
            piece2 = piece2.upper()
        else:
            piece = piece.lower()
            piece2 = piece2.lower()
        return tuple([piece,row1,col1,row2,col2,piece2])
    
    def add_piece(self,order):
        [piece,row,col] = self.position_to_index(order)
        if 0 <= row < 8 and 0 <= col <8 and piece in ['R','N','B','Q','K','k']:
            if piece == 'K' and 'K' in self.pieces_white:
                print('Only one white king can be added')
            elif piece == 'k' and 'k' in self.pieces_black:
                print('Only one black king can be added')
            elif self.board[row][col] == '.':
                self.board[row][col] = piece
                self.eval_board()
                if not self.checked:
                    self.print_black_board()
                    self.print_board()
                    if piece.islower():
                        self.pieces_black.append(piece)
                    else:
                        self.pieces_white.append(piece)
                    print('Successfully added '+ order)
                else:
                    self.board[row][col] = '.'
                    self.eval_board()
                    print('Black cannot be checked at start')
            else:
                print('Piece already exists at that position')
        else:
            print('Invalid position or piece')
    
    def eval_board(self):
        possible_moves_white = []
        possible_moves_black = []
        new_black_board = [['.' for _ in range(8)] for _ in range(8)]
        new_white_board = [['.' for _ in range(8)] for _ in range(8)]
        # Mark black king's position
        for row in range(8):
            for col in range(8):
                if self.board[row][col] == 'k':
                    bkr = row
                    bkc = col
        # Mark black king's surrounding
        moveset = [[_,__] for _ in [1,-1] for __ in [1,-1]]
        for m in moveset:
            if 0 <= bkr + m[0] < 8 and 0 <= bkc + m[1] < 8:
                new_white_board[bkr+m[0]][bkc+m[1]] = 'o'
        
        for row in range(8):
            for col in range(8):
                if self.board[row][col] == '.':
                    continue

                elif self.board[row][col] == 'P':
                    # Pawn
                    if 0 <= row + 1 < 7:
                        if self.board[row+1][col] == '.':
                            possible_moves_white.append(tuple(['P',row,col,row+1,col,'P']))
                        if self.board[row+1][col+1].islower() and 0 <= col + 1 < 8:
                            possible_moves_white.append(tuple(['P',row,col,row+1,col+1,'P']))
                        if self.board[row+1][col-1].islower() and 0 <= col - 1 < 8:
                            possible_moves_white.append(tuple(['P',row,col,row+1,col-1,'P']))
                    elif row + 1 == 7:
                        #promote
                        if self.baord[row+1][col] == '.':
                            possible_moves_white.append(tuple(['P',row,col,row+1,col,'N']))
                            possible_moves_white.append(tuple(['P',row,col,row+1,col,'Q']))
                        if self.board[row+1][col+1].islower() and 0 <= col + 1 < 8:
                            possible_moves_white.append(tuple(['P',row,col,row+1,col+1,'N']))
                            possible_moves_white.append(tuple(['P',row,col,row+1,col+1,'Q']))
                        if self.board[row+1][col-1].islower() and 0 <= col - 1 < 8:
                            possible_moves_white.append(tuple(['P',row,col,row+1,col-1,'N']))
                            possible_moves_white.append(tuple(['P',row,col,row+1,col-1,'Q']))
                    if 0 <= row + 1 < 8:
                        if col + 1 < 8:
                            new_black_board[row+1][col+1] = 'x'
                        if col - 1 >= 0:
                            new_black_board[row+1][col-1] = 'x'

                elif self.board[row][col] == 'R':
                    # Rook
                    for dir in [1,-1]:
                        i = 1
                        n_row = row + i*dir
                        n_col = col
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves_white.append(tuple(['R',row,col,n_row,n_col]))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col
                        i = 1
                        n_row = row
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves_white.append(tuple(['R',row,col,n_row,n_col]))
                            else:
                                break
                            i += 1
                            n_row = row
                            n_col = col + i*dir

                elif self.board[row][col] == 'N':
                    moveset = [[_,__] for _ in [2,1,-1,-2] for __ in [2,1,-1,-2] if abs(_)!=abs(__)]
                    for m in moveset:
                        n_row = row + m[0]
                        n_col = col + m[1]
                        if 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves_white.append(tuple(['N',row,col,n_row,n_col]))

                elif self.board[row][col] == 'B':
                    for dir in [1,-1]:
                        i = 1
                        n_row = row + i*dir
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves_white.append(tuple(['B',row,col,n_row,n_col]))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col + i*dir
                        i = 1
                        n_row = row + i*dir
                        n_col = col - i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves_white.append(tuple(['B',row,col,n_row,n_col]))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col - i*dir

                elif self.board[row][col] == 'Q':
                    for dir in [1,-1]:
                        i = 1
                        n_row = row + i*dir
                        n_col = col
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves_white.append(tuple(['R',row,col,n_row,n_col]))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col
                        i = 1
                        n_row = row
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves_white.append(tuple(['R',row,col,n_row,n_col]))
                            else:
                                break
                            i += 1
                            n_row = row
                            n_col = col + i*dir
                        i = 1
                        n_row = row + i*dir
                        n_col = col + i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves_white.append(tuple(['B',row,col,n_row,n_col]))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col + i*dir
                        i = 1
                        n_row = row + i*dir
                        n_col = col - i*dir
                        while 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper():
                                possible_moves_white.append(tuple(['B',row,col,n_row,n_col]))
                            else:
                                break
                            i += 1
                            n_row = row + i*dir
                            n_col = col - i*dir

                elif self.board[row][col] == 'K':
                    moveset = [[_,__] for _ in [1,-1] for __ in [1,-1]]
                    for m in moveset:
                        n_row = row + m[0]
                        n_col = col + m[1]
                        if 0 <= n_row < 8 and 0 <= n_col < 8:
                            new_black_board[n_row][n_col] = 'x'
                            if not self.board[n_row][n_col].isupper() and new_white_board[n_row][n_col] != 'o':
                                possible_moves_white.append(tuple(['K',row,col,n_row,n_col]))

        moveset = [[_,__] for _ in [1,-1] for __ in [1,-1]]
        for m in moveset:
            n_row = bkr + m[0]
            n_col = bkc + m[1]
            if 0 <= n_row < 8 and 0 <= n_col < 8:
                if not self.board[n_row][n_col].isupper() and new_black_board[n_row][n_col] != 'x':
                    possible_moves_black.append(tuple(['k',bkr,bkc,n_row,n_col]))
        self.board_black = new_black_board
        self.checked = self.board_black[self.black_king[0]][self.black_king[1]] == 'x'
        if self.checked:
            self.board_black[self.black_king[0]][self.black_king[1]] = '*'
        else:
            self.board_black[self.black_king[0]][self.black_king[1]] = 'k'
        if not self.checked and len(possible_moves_black) == 0 and self.current_player == 'white':
            self.score = -10000
        else:
            self.score = 8 - len(possible_moves_black)
        self.stalemated = not self.checked and len(possible_moves_black) == 0
        self.checkmated = self.checked and len(possible_moves_black) == 0

    def make_move(self, move):
        from_row, from_col, to_row, to_col = move
        piece = self.board[from_row][from_col]
        self.board[from_row][from_col] = '.'
        self.board[to_row][to_col] = piece
        if self.current_player == 'white':
            self.current_player = 'black'
        else:
            self.current_player = 'white'


    def get_random_move(self):
        possible_moves = []
        for row in range(8):
            for col in range(8):
                if self.board[row][col].isupper() and self.current_player == 'white' :
                    for to_row in range(8):
                        for to_col in range(8):
                            if self.is_valid_move((row, col, to_row, to_col)):
                                possible_moves.append((row, col, to_row, to_col))

                elif self.board[row][col].islower() and self.current_player == 'black':
                    for to_row in range(8):
                        for to_col in range(8):
                            if self.is_valid_move((row, col, to_row, to_col)):
                                possible_moves.append((row, col, to_row, to_col))

        if not possible_moves:
            return None

        return random.choice(possible_moves)

    def is_valid_move(self, move):
        from_row, from_col, to_row, to_col = move
        if not (0 <= from_row < 8 and 0 <= from_col < 8 and 0 <= to_row < 8 and 0 <= to_col < 8):
            return False

        piece = self.board[from_row][from_col]
        if piece == '.':
            return False

        if (self.current_player == 'white' and piece.islower()) or (self.current_player == 'black' and piece.isupper()):
            return False

        if piece == 'P':
            if to_row == from_row + 1 and to_col == from_col and self.board[to_row][to_col] == '.':
                return True
            # Add more pawn move logic here
        return False  # Placeholder for other piece moves

def main():

    # initiate game with black king position
    game = ChessGame('a6')
    game.add_piece('Qe4')
    # game.print_board()
    # while True:
    #     move = game.get_random_move()
    #     if move is None:
    #         print("No more moves possible. Game Over.")
    #         break
    #     game.make_move(move)
    #     game.print_board()


if __name__ == "__main__":
    main()

  +---+---+---+---+---+---+---+---+
8 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
7 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
6 | k | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
5 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
4 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
3 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
2 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
1 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h
  +---+---+---+---+---+---+---+---+
8 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
7 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
6 | k | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
5 | . | . | . | . | . | . | . | . | 
  +---+---+---+---+---+---+---+---+
4 | . | . | . | . 